# LG-CoTrain: All Disasters Experiment

This notebook runs the **LG-CoTrain** co-training pipeline across **all 10 disaster
events**, each with **4 budgets x 3 seed sets = 12 experiments**, for a total of
**120 experiments**.

### Resume Support (two levels)

1. **Event level**: Before starting an event, we check if all 12 `metrics.json`
   files already exist. If so, the entire event is skipped.
2. **Experiment level**: Within each event, `run_all_experiments` skips individual
   `(budget, seed_set)` combinations that already have `metrics.json`.

This means you can safely **restart the notebook after a crash** and it will
resume from where it left off.

### Events

| # | Event |
|---|-------|
| 1 | california_wildfires_2018 |
| 2 | canada_wildfires_2016 |
| 3 | cyclone_idai_2019 |
| 4 | hurricane_dorian_2019 |
| 5 | hurricane_florence_2018 |
| 6 | hurricane_harvey_2017 |
| 7 | hurricane_irma_2017 |
| 8 | hurricane_maria_2017 |
| 9 | kaikoura_earthquake_2016 |
| 10 | kerala_floods_2018 |

In [1]:
import json
import statistics
import sys
import time
from pathlib import Path

def _find_repo_root(marker: str = "lg_cotrain") -> Path:
    for candidate in [Path().resolve()] + list(Path().resolve().parents):
        if (candidate / marker).is_dir():
            return candidate
    raise RuntimeError(
        f"Cannot find repo root: no ancestor directory contains '{marker}/'. "
        "Run the notebook from inside the repository."
    )

repo_root = _find_repo_root()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import matplotlib.pyplot as plt
import numpy as np

from lg_cotrain.run_all import BUDGETS, SEED_SETS, run_all_experiments, format_summary_table

DATA_ROOT = str(repo_root / "data")
RESULTS_ROOT = str(repo_root / "results")

print(f"Repo root: {repo_root}")
print(f"Data root: {DATA_ROOT}")
print(f"Results root: {RESULTS_ROOT}")
print(f"Budgets: {BUDGETS}")
print(f"Seed sets: {SEED_SETS}")
print(f"Experiments per event: {len(BUDGETS) * len(SEED_SETS)}")

Repo root: D:\Workspace\Co-Training
Data root: D:\Workspace\Co-Training\data
Results root: D:\Workspace\Co-Training\results
Budgets: [5, 10, 25, 50]
Seed sets: [1, 2, 3]
Experiments per event: 12


In [2]:
def is_event_complete(event, results_root):
    """Check if all 12 metrics.json files exist for an event."""
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(results_root) / event / f"{budget}_set{seed_set}" / "metrics.json"
            if not path.exists():
                return False
    return True

# Discover events from data directory
data_dir = Path(DATA_ROOT) / "original"
all_events = sorted(p.name for p in data_dir.iterdir() if p.is_dir())

completed_events = [e for e in all_events if is_event_complete(e, RESULTS_ROOT)]
pending_events = [e for e in all_events if e not in completed_events]

print(f"Found {len(all_events)} events total")
print(f"  Completed: {len(completed_events)} ({len(completed_events) * 12} experiments)")
print(f"  Pending:   {len(pending_events)} (up to {len(pending_events) * 12} experiments)")

if completed_events:
    print(f"\nCompleted events (will be skipped):")
    for e in completed_events:
        print(f"  - {e}")

if pending_events:
    print(f"\nPending events (will be run):")
    for e in pending_events:
        print(f"  - {e}")

Found 10 events total
  Completed: 0 (0 experiments)
  Pending:   10 (up to 120 experiments)

Pending events (will be run):
  - california_wildfires_2018
  - canada_wildfires_2016
  - cyclone_idai_2019
  - hurricane_dorian_2019
  - hurricane_florence_2018
  - hurricane_harvey_2017
  - hurricane_irma_2017
  - hurricane_maria_2017
  - kaikoura_earthquake_2016
  - kerala_floods_2018


## Running Experiments

For each pending event, we call `run_all_experiments` which runs all 12
`(budget, seed_set)` combinations. Individual experiments that already have
`metrics.json` are automatically skipped (useful if the notebook crashed
mid-event).

After each event completes, its summary table is printed immediately so
results are visible even if the notebook is interrupted later.

In [ ]:
class ProgressTracker:
    """Track global progress across all experiments."""

    def __init__(self, total, already_done, start_time):
        self.total = total
        self.done = already_done
        self.start_time = start_time

    def update(self, event, budget, seed_set, status):
        self.done += 1
        elapsed = time.time() - self.start_time
        pct = 100.0 * self.done / self.total
        elapsed_h = elapsed / 3600

        # ETA: based on rate of non-skipped work so far
        remaining = self.total - self.done
        if elapsed > 0 and self.done > 0:
            eta_h = (elapsed / self.done) * remaining / 3600
        else:
            eta_h = 0

        print(
            f"[PROGRESS] {self.done}/{self.total} ({pct:.1f}%)"
            f" | Elapsed: {elapsed_h:.2f}h | ETA: {eta_h:.2f}h"
        )

# Count already-completed experiments (from previous runs)
already_done = sum(
    1
    for e in all_events
    for b in BUDGETS
    for s in SEED_SETS
    if (Path(RESULTS_ROOT) / e / f"{b}_set{s}" / "metrics.json").exists()
)
total_experiments = len(all_events) * len(BUDGETS) * len(SEED_SETS)

print(f"Total experiments: {total_experiments}")
print(f"Already completed: {already_done}")
print(f"Remaining: {total_experiments - already_done}")

all_event_results = {}
overall_start = time.time()
tracker = ProgressTracker(total_experiments, already_done, overall_start)

# Run pending events
for i, event in enumerate(pending_events, 1):
    print(f"\n{'=' * 60}")
    print(f"Event {i}/{len(pending_events)}: {event}")
    print(f"{'=' * 60}")

    results = run_all_experiments(
        event,
        data_root=DATA_ROOT,
        results_root=RESULTS_ROOT,
        _on_experiment_done=tracker.update,
    )
    all_event_results[event] = results

    print()
    print(format_summary_table(results, event))

# Load results for already-completed events
for event in completed_events:
    results = []
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(RESULTS_ROOT) / event / f"{budget}_set{seed_set}" / "metrics.json"
            with open(path) as f:
                results.append(json.load(f))
    all_event_results[event] = results

overall_elapsed = time.time() - overall_start
print(f"\n{'=' * 60}")
print(f"All events done in {overall_elapsed / 3600:.2f}h")
print(f"Total events with results: {len(all_event_results)}")

Total experiments: 120
Already completed: 0
Remaining: 120

Event 1/10: california_wildfires_2018


D:\Workspace\Co-Training\co-training-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[1/12] budget=5, seed=1 -- starting...


2026-02-17 02:52:46,648 - lg_cotrain - INFO - Starting LG-CoTrain: event=california_wildfires_2018, budget=5, seed_set=1
2026-02-17 02:52:46,686 - lg_cotrain - INFO - Detected 10 classes for event california_wildfires_2018: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'missing_or_found_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-17 02:52:46,699 - lg_cotrain - INFO - D_l1: 30, D_l2: 20, D_LG: 5113
2026-02-17 02:52:46,702 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1038.50it/s, Materializing param=bert.pooler.dense.weight]
2026-02-17 02:53:05,658 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.0981, mean_prob2=0.1025
2026-02-17 02:53:23,181 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.0997, mean_prob2=0.0991
2

[1/12] budget=5, seed=1 -- done (macro_f1=0.6162)
[PROGRESS] 1/120 (0.8%) | Elapsed: 0.25h | ETA: 29.44h
[2/12] budget=5, seed=2 -- starting...


...

2026-02-17 13:21:17,418 - lg_cotrain - INFO - Starting LG-CoTrain: event=hurricane_florence_2018, budget=50, seed_set=3
2026-02-17 13:21:17,462 - lg_cotrain - INFO - Detected 9 classes for event hurricane_florence_2018: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-17 13:21:17,471 - lg_cotrain - INFO - D_l1: 219, D_l2: 219, D_LG: 3946
2026-02-17 13:21:17,473 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|████████████████| 199/199 [00:00<00:00, 991.11it/s, Materializing param=bert.pooler.dense.weight]
2026-02-17 13:21:34,578 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1135, mean_prob2=0.1222
2026-02-17 13:21:50,322 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1147, mean_prob2=0.1212
2026-02-17 13:22:06,100 - lg_c

[12/12] budget=50, seed=3 -- done (macro_f1=0.6678)
[PROGRESS] 60/120 (50.0%) | Elapsed: 10.69h | ETA: 10.69h

Batch complete: 12 ran, 0 skipped, 0 failed (8757.1s total)

=== Results for hurricane_florence_2018 ===

Budget    Seed 1              Seed 2              Seed 3                  Mean       Std
           ErrR%  MacF1   ErrR%  MacF1   ErrR%  MacF1     ErrR%     MacF1
-------------------------------------------------------------------------
     5     25.38 0.6458   25.79 0.6345   25.30 0.6475  25.49+/-0.26   0.6426+/-0.0070
    10     24.82 0.6551   26.51 0.6475   25.71 0.6409  25.68+/-0.85   0.6478+/-0.0071
    25     24.42 0.6732   24.42 0.6696   25.71 0.6607  24.85+/-0.74   0.6678+/-0.0065
    50     28.20 0.6612   23.45 0.6821   26.11 0.6678  25.92+/-2.38   0.6704+/-0.0107

Event 6/10: hurricane_harvey_2017
[1/12] budget=5, seed=1 -- starting...


2026-02-17 13:33:57,617 - lg_cotrain - INFO - Starting LG-CoTrain: event=hurricane_harvey_2017, budget=5, seed_set=1
2026-02-17 13:33:57,714 - lg_cotrain - INFO - Detected 9 classes for event hurricane_harvey_2017: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-17 13:33:57,727 - lg_cotrain - INFO - D_l1: 27, D_l2: 18, D_LG: 6333
2026-02-17 13:33:57,730 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|████████████████| 199/199 [00:00<00:00, 887.76it/s, Materializing param=bert.pooler.dense.weight]
2026-02-17 13:34:22,889 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1226, mean_prob2=0.1284
2026-02-17 13:34:45,974 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1223, mean_prob2=0.1253
2026-02-17 13:35:08,982 - lg_cotrain 

[1/12] budget=5, seed=1 -- done (macro_f1=0.6143)
[PROGRESS] 61/120 (50.8%) | Elapsed: 10.99h | ETA: 10.63h
[2/12] budget=5, seed=2 -- starting...


2026-02-17 13:51:57,818 - lg_cotrain - INFO - Starting LG-CoTrain: event=hurricane_harvey_2017, budget=5, seed_set=2
2026-02-17 13:51:57,858 - lg_cotrain - INFO - Detected 9 classes for event hurricane_harvey_2017: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-17 13:51:57,869 - lg_cotrain - INFO - D_l1: 27, D_l2: 18, D_LG: 6333
2026-02-17 13:51:57,871 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1028.06it/s, Materializing param=bert.pooler.dense.weight]
2026-02-17 13:52:21,317 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1109, mean_prob2=0.1146
2026-02-17 13:52:43,384 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1126, mean_prob2=0.1151
2026-02-17 13:53:05,640 - lg_cotrain 

## Cross-Disaster Results

We now aggregate results across all 10 events to compare how the pipeline
performs on different disaster types and how performance scales with the
labeled data budget.

In [ ]:
# Build cross-disaster summary: event -> budget -> mean macro-F1
summary = {}
for event in sorted(all_event_results.keys()):
    results = all_event_results[event]
    by_budget = {b: [] for b in BUDGETS}
    for r in results:
        if r is not None:
            by_budget[r["budget"]].append(r)
    summary[event] = {}
    for b in BUDGETS:
        f1s = [r["test_macro_f1"] for r in by_budget[b]]
        errs = [r["test_error_rate"] for r in by_budget[b]]
        summary[event][b] = {
            "f1_mean": statistics.mean(f1s) if f1s else None,
            "f1_std": statistics.stdev(f1s) if len(f1s) >= 2 else None,
            "err_mean": statistics.mean(errs) if errs else None,
            "err_std": statistics.stdev(errs) if len(errs) >= 2 else None,
            "n_seeds": len(f1s),
        }

# Print grand summary table
header = f"{'Event':<35}"
for b in BUDGETS:
    header += f" | B={b:<11}"
print(header)
print("-" * len(header))

for event in sorted(summary.keys()):
    row = f"{event:<35}"
    for b in BUDGETS:
        s = summary[event][b]
        if s["f1_mean"] is not None and s["f1_std"] is not None:
            row += f" | {s['f1_mean']:.3f}+/-{s['f1_std']:.3f}"
        elif s["f1_mean"] is not None:
            row += f" | {s['f1_mean']:.3f}      "
        else:
            row += f" | {'N/A':<11}"
    print(row)

In [ ]:
# Line plot: Macro-F1 by budget, one line per event
fig, ax = plt.subplots(figsize=(10, 6))

for event in sorted(summary.keys()):
    means = [summary[event][b]["f1_mean"] or 0 for b in BUDGETS]
    stds = [summary[event][b]["f1_std"] or 0 for b in BUDGETS]
    ax.errorbar(BUDGETS, means, yerr=stds, marker="o", capsize=3, label=event)

ax.set_xlabel("Budget (labeled samples per class)")
ax.set_ylabel("Test Macro-F1 (mean +/- std across seeds)")
ax.set_title("LG-CoTrain Performance Across Disasters")
ax.set_xticks(BUDGETS)
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: events (rows) x budgets (columns), colored by mean macro-F1
events_sorted = sorted(summary.keys())
heatmap_data = np.zeros((len(events_sorted), len(BUDGETS)))

for i, event in enumerate(events_sorted):
    for j, b in enumerate(BUDGETS):
        val = summary[event][b]["f1_mean"]
        heatmap_data[i, j] = val if val is not None else 0

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(heatmap_data, cmap="YlOrRd", aspect="auto")

ax.set_xticks(range(len(BUDGETS)))
ax.set_xticklabels([f"B={b}" for b in BUDGETS])
ax.set_yticks(range(len(events_sorted)))
ax.set_yticklabels(events_sorted, fontsize=9)
ax.set_title("Mean Test Macro-F1 by Event and Budget")

# Annotate cells with values
for i in range(len(events_sorted)):
    for j in range(len(BUDGETS)):
        val = heatmap_data[i, j]
        color = "white" if val > 0.6 else "black"
        ax.text(j, i, f"{val:.3f}", ha="center", va="center", color=color, fontsize=9)

fig.colorbar(im, ax=ax, label="Macro-F1")
plt.tight_layout()
plt.show()

## Summary

This notebook ran **120 experiments** (10 events x 4 budgets x 3 seed sets)
using the LG-CoTrain pipeline.

### Resume support
- Events with all 12 `metrics.json` files are automatically skipped
- Within partially-completed events, individual experiments with existing
  results are skipped
- Safe to restart after crashes

### CLI equivalent
```bash
# Run all 12 experiments for a single event
python -m lg_cotrain.run_all --event california_wildfires_2018
python -m lg_cotrain.run_all --event canada_wildfires_2016
# ... etc for all 10 events
```